In [ ]:
from cycler import cycler
from datetime import datetime
from duckdb.typing import TIMESTAMP_TZ, DOUBLE, BOOLEAN
from IPython.display import Markdown

import duckdb
import locale
import pandas as pd
import numpy as np
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.lines as mlines
import math
from scipy.optimize import curve_fit
import suncalc
import warnings

locale.setlocale(locale.LC_ALL, 'de_DE')

tz='Europe/Berlin'

# Create a connection to your PV database. 
# Make sure you installed the schema as described in the README of the pv project.
db = duckdb.connect('pv.db')

v = db.execute('SELECT year(max(measured_on)) FROM measurements').fetchone()[0]

if v is not None:
    dt = datetime(v, 1, 1)
else:
    dt = datetime.today()
    dt = datetime(dt.year, 1, 1)
    
prop_cycle = plt.rcParams['axes.prop_cycle']

# Place of installation, used on several occasions
lat_long = db.execute('FROM v_place_of_installation').df()
lat, long = lat_long['lat'][0], lat_long['long'][0]

# Helper functions
localize_tz = lambda v: v.tz_localize(tz=tz)
to_degree = lambda v: math.degrees(v) if v > 0 else 0

In [ ]:
%%html
<style>
div.jp-OutputArea-executeResult > div.jp-RenderedMarkdown {
    padding-left: var(--jp-code-padding);
}
.jp-Notebook {
    max-width: 1280px;
    margin-left: auto;
    margin-right: auto;
}
</style>

# Statistiken und Gedanken zur Photovoltaikanlage

In [ ]:
now = datetime.now()
Markdown(f"""
Letzte Aktualisierung: _{now.strftime('%x')}, {now.strftime('%X')}_.

Die Quelltexte, inklusiver aller Datenbankstrukturen und Abfragen sowie das Jupyiter Notebook mit dem diese Seite generiert wurde sind unter Apache 2 License in meinem PV Repository veröffentlicht: [michael-simons/pv](https://github.com/michael-simons/pv).
Diese Seite dient als kurzer Überblick über unsere Photovoltaikanlage in Aachen. Mit der Planung begannen wir im September 2022, Ende April 2023 speiste der Wechselrichter das erste Mal Strom in den Hauskreislauf und das Stromnetz, Ende Mai habe ich zum ersten Mal sinnvoll bilanzierte Werte aus dem Monitoringdevice auslesen können.
Seit Ende Juni haben wir einen 2-Wegezähler und warten darauf, dass wir endlich durch die Einspeisevergütung reich werden ;)
""")

## Warum diese Seite?

Ursprünglich hatte ich nicht vor, allzu viel Arbeit und Ehrgeiz in die Analyse der Daten der PV-Anlage zu stecken, weder in Charts noch irgendwelche anderen Satellitensoftware. Immerhin hatten wir doch bereits einen Energiemanager mitgekauft, der irgendwann einmal Batterie- und Wallboxsteuerung übernehmen soll. Es stellte sich aber schnell heraus, dass—wie üblich—Smartdevices nicht wirklich smart sind, Konfiguration auch für Profis schwierig ist und am Ende hab ich doch wieder selber etwas gebaut: Das aktuelle Setup liest die Daten des Wechselrichters direkt über [SunSpec Modbus](https://sunspec.org/sunspec-modbus-specifications/) aus und schreibt sie in eine [DuckDB Datenbank](https://duckdb.org). Den zu diesem Zweck programmierte  [Logger](https://github.com/michael-simons/pv/tree/main/logger) basiert auf dem [Energy systems reading toolkit](https://energy.basjes.nl) von [Niels Basjes](https://mastodon.basjes.nl/@niels). Bedankt, Niels.

DuckDB ist eine sehr coole In-Prozess Datenbank mit dem Schwerpunkt [OLAP](https://de.wikipedia.org/wiki/Online_Analytical_Processing). Das Datenbankdesign dieser Anwendung folgt einer Teilmenge des [pink database design](https://www.salvis.com/blog/2018/07/18/the-pink-database-paradigm-pinkdb/#feature_3). Anstatt alle Queries direkt in einer Anwendung zu schreiben (oder gar ein ORM Framework zu nutzen und Statistiken in der Anwendung zu berechnen), habe ich eine ganze handvoll Views erstellt. Diese repräsentieren sozusagen die öffentliche API des Schemas. Sie sind natürlich auch im Repository gespeichert und liegen im [Schema-Ordner](https://github.com/michael-simons/pv/tree/main/schema) (Alle Dateien die mit `R__` anfangen).

## Warum die Photovoltaikanlage?

Wir wohnen seit 2010 im selben Ort und unsere Bezugskosten sind bis Ende 2019 mehr oder weniger kontinuierlich, aber vorhersagbar gestiegen. Mit der Pandemie, dem Krieg in der Ukraine, den Kosten des Klimawandels und schlussendlich der daraus steigenden Inflation wurden die Preise danach immer unkalkulierbarer. Langfristig ist unser Ziel, zumindest über einen Teil des Jahres stabile Stromkosten zu haben *und* dieses Haus nachhaltiger zu bewirtschaften.

Im folgenden ein Diagramm unserer Beschaffungskosten seit 2010.

In [ ]:
buying_prices = db.execute('SELECT valid_from AS Jahr, net FROM v__buying_prices').df()
imported = db.execute("SELECT date_trunc('year', period_end) as Jahr, import FROM official_measurements ORDER by period_start ASC").df()

imported_with_price = pd.merge_asof(buying_prices, imported, on='Jahr')

fig, ax = plt.subplots(figsize=plt.figaspect(1/3))
imported_with_price.plot(x='Jahr', y='net', ylabel='ct/kWh', label='Preis (links)', ax=ax)
imported_with_price.plot(x='Jahr', y='import', ylabel='kWh', label='Arbeit (rechts)', ax=ax, secondary_y=True, mark_right=False)
plt.title("Beschaffungskosten seit 2010")

plt.show()

Während die Menge bezogener Energie über die Jahre verhältnismäßig konst geblieben ist (2018 habe ich angefangen, 100% Remote zu Arbeite und seit ein paar Jahren spielen die Kinder mehr Computer als ich an einem arbeite, aber der Mehrbetrag ist vernachlässigbar). Der Bezugspreis hingegen ist seit 2010 um **75%** gestiegen. 

## Die Anlage

Wir haben eine Anlage mit einer Peak-Leistung von 10.53kWp aufgebaut in einer aufgeteilten Ost/West Aufstellung. Ingesamt sind 26 [Solarwatt Glass-Glass "Panel Vision AM 4.0"](https://solarwatt.canto.global/direct/document/2bp9ip7a492p51429ek8qdr706/HPyRw7XsY0A3hu2B1c6SpNGxiOc/original?content-type=application%2Fpdf&name=Datenblatt+SOLARWATT+Panel+vision+AM+4.0+pure+de.pdf) Module mit einem Peak-Output von 405Wp pro Modul verbaut. Alle Module sind "Plus-Auswahl", d.h. 405Wp sind garantiert.

* 14 in östlicher Richtung
* 12 in westlicher Richtung

Als Wechselrichter kommt ein Kaco [blueplanet 10.0 NX3 M2](https://kaco-newenergy.com/de/produkte/blueplanet-3.0-20.0-NX3-M2/) zum Einsatz, eines der wenigen Modelle die unser Installateur beziehungsweise [Solarwatt](https://www.solarwatt.de) liefern konnte. Ebenfalls von Solarwatt stammt der "Energymanager", der ist im ersten Monat soviel "Freude" bereitet hat. Immerhin, die Hotline war bemüht und hilfreich.

In [ ]:
overall_production = db.execute('SELECT total, total_yield, daily_median FROM v_overall_production').fetchone()
Markdown(f"""
Mit der Anlage haben wir seit Inbetriebname ingesamt **{overall_production[0]:.0f} kWh** produziert, was einem Ertrag von **{overall_production[1]:.0f} kWh/kWp** entspricht.
Im Schnitt werden täglich **{overall_production[2]:.0f} kWh** erzeugt.
Der mögliche Peak-Output von 10.53 kWp ist deutlich mehr als wie wir täglich tatsächlich verbrauchen, aber mit Hinblick auf Autarkie in der Zukunft und weitere Anwendungen, wollten wir lieber sicher gehen. Bis jetzt steht noch offen, ob wir unser mehr als 10 Jahre altes Auto mit einem elektrischen ersetzen oder eine stationäre Batterie, eine Wärmepumpe oder beides kaufen.
""")

## Ergebnisse

### Zusammenfassung

Bevor wir ins Detail gehen, hier die jährliche Zusammenfassung. Die Berechnung der CO<sub>2</sub>-Ersparnis basiert auf dem vom Umweltbundesamt bereitgestellten Bericht über die ["Entwicklung der spezifischen Treibhausgas-Emissionen des deutschen Strommix in den Jahren 1990 - 2023"](https://www.umweltbundesamt.de/publikationen/entwicklung-der-spezifischen-treibhausgas-10) sowie dem Eigenverbrauch aus der PV-Anlage:

In [ ]:
new_names_production = {
    'worst': 'Schlechtester Tag',
    'best': 'Bester Tag',
    'daily_avg': 'Durchschnitt',
    'daily_median': 'Median',
    'total': 'Gesamtproduktion',
    'total_yield': 'Ertrag (kWh/kWp)',
    'year': 'Jahr'
}
yearly_production = db.execute('FROM v_yearly_production') \
  .df().rename(columns=new_names_production) \
  .set_index('Jahr')

new_names_quotes = {'internal_consumption': 'Eigenverbrauchsanteil (%)', 'autarchy': 'Autarkiegrad', 'year': 'Jahr'}
yearly_internal_consumption_share = db.execute('FROM v_yearly_internal_consumption_share')\
  .df().fillna(0) \
  .rename(columns=new_names_quotes) \
  .set_index('Jahr')

yearly_accumulated_co2_savings = db.execute('FROM v_accumulated_yearly_co2_savings')\
  .df().fillna(0) \
  .rename(columns={'year': 'Jahr', 'total': 'CO2-Ersparnis (kg)'})\
  .set_index('Jahr')

yearly_summary = yearly_production \
  .join(yearly_internal_consumption_share) \
  .join(yearly_accumulated_co2_savings)

def add_link(val):
    return val if val == dt.year else f'<a href="./{val}.html">{val}</a>'

yearly_summary.style \
  .format(precision = 2) \
  .format_index(add_link)

Ein Flug nach Mallorca verursacht laut WWF-Berechnungen wohl 925 Killogramm CO<sub>2</sub> pro Kopf (Quelle: [Der Spiegel](https://www.spiegel.de/reise/aktuell/klimabilanz-mallorca-urlaub-so-schaedlich-wie-ein-jahr-autofahren-a-642221.html)). Das nur zum Vergleich.

In [ ]:
period = db.execute('SELECT min(measured_on) AS min, max(measured_on) AS max FROM measurements').df()
Markdown(f"""
### Erzeugung

Die historischen Wetterdaten (unter anderem Sonneneinstrahlung in W/m<sup>2</sup>) für die folgenden Diagramme wurden über die wunderbare [Open-Meteo API](https://open-meteo.com/) bezogen. Vielen Dank dafür.
Aktuell liegen Messungen vom **{period['min'][0].strftime('%x')}** bis zum **{period['max'][0].strftime('%x')}** vor. In dieser Zeit haben wir folgende Monatsdurchschnittswerte ermittelt (Es werden nur "volle" Monate betrachtet):
""")

In [ ]:
average_per_month = db.execute('FROM v_average_production_per_month').df()
average_per_month['month'] = average_per_month['month'].apply(lambda i: datetime(2023, i, 1).strftime('%B'))

colors = prop_cycle.by_key()['color']
fig, ax = plt.subplots(figsize=plt.figaspect(1/3))

average_per_month.plot(xlabel='Monat', label='Produktion', x='month', y='production', ylabel='kWh', kind='bar', ax=ax, color=colors[0])
average_per_month.plot(xlabel='Monat', label='Mittlere Sonneneinstrahlung je m²', x='month', y='shortwave_radiation', ax=ax, mark_right=False, color=colors[1])

plt.title("Durchschnittliche Erzeugung pro Monat")
plt.show()

In [ ]:
start = datetime(dt.year, 3, 21)
end = start + pd.DateOffset(months=6)
df = db.execute('FROM v_weekly_quartiles WHERE sow BETWEEN ? AND ?', [start, end]).df()

if not df.empty:
  display(Markdown(f"""
  Die Darstellung der wöchentlichen Erzeugung berücksichtigt Jahreszeiten und tägliches Wetter etwas besser als eine pauschale Darstellung der durchschnittlichen Erzeugung pro Monat.
  Im Chart ist zusätzliche die durchschnittliche Sonneneinstrahlung in kWh/m² sowie die Wolkenabdeckung niedriger Höhe in [Okta](https://de.wikipedia.org/wiki/Bewölkung) eingetragen:
  """))

  df = df.rename(columns={"min": "whislo", "max": "whishi"})
  df['label'] = df['week'].apply(lambda w: "KW" + str(w))
  df[["q1", "med", "q3"]] = df["quartiles"].to_list()

  _, ax = plt.subplots(figsize=plt.figaspect(1/3))
  ax.bxp(df.to_dict(orient='records'), showfliers=False)
  ax.set_ylabel("kWh")

  indexes = np.arange(len(df['shortwave_radiation'])) + 1
  plt.plot(indexes, df['shortwave_radiation'])
    
  for xitem,yitem in np.nditer([indexes, df['shortwave_radiation']]):
    plt.annotate(df['cloud_coverage'][xitem-1], (xitem,yitem), textcoords="offset points", xytext=(0,-1), ha="center", va="baseline", bbox=dict(facecolor='white', edgecolor='white', boxstyle="square,pad=0.15"))
    
  plt.title(f"Wöchentliche Erzeugung von März bis September in {dt.year}")
  plt.show()

In [ ]:
max_peak = db.execute('SELECT round(production) as max_peak FROM v_peaks').df()['max_peak'][1];
Markdown(f"""
Die Wahrscheinlichkeit, dass wir jemals an die tatsächlich installierte Peak-Leistung herankommen werden, ist denkbar gering. 
Der bisher höchste, gemessene Wert in einer Viertelstunde waren **{max_peak} W**.
Viel interessanter—und relevanter—ist jedoch die Tatsache, dass wir mit der gewählten Dachbelegung den ganzen Tag über einen brauchbaren Output haben.
Das kann als Durchschnittswert pro Stunde visualisiert werden:
""")

In [ ]:
average_per_hour = db.execute('FROM v_average_production_per_hour').df()

colors = prop_cycle.by_key()['color']
fig, ax = plt.subplots(figsize=plt.figaspect(1/3))

average_per_hour.plot(xlabel='Stunde', label='Produktion', x='hour', y='production', ylabel='kWh', kind='bar', ax=ax, color=colors[0])
average_per_hour.plot(xlabel='Stunde', label='Mittlere Sonneneinstrahlung je m²', x='hour', y='shortwave_radiation', ax=ax, mark_right=False, color=colors[1])

plt.title("Durchschnittliche Erzeugung pro Stunde")
plt.show()

Ich mochte die Visualisierung von [Oli](https://social.tchncs.de/@oli) im Beitrag ["Ein Jahr Photovoltaik: Tolle Dinge, die man mit SQL machen kann"](https://tonick.net/p/2022/12/ein-jahr-photovoltaik/) so sehr, dass ich sie auch haben wollte. Und zwar die durchschnittliche Erzeugung pro Monat und Stunden, quasi die Kombination der beiden oberen Balkendiagramme. Diese Heatmap sieht tatsächlich nicht nur schön aus, sie ist auch informativ und gibt direkt eine Idee, zu welchem Teil des Jahres der Eigennutzungsgrad sehr hoch sein wird und wann nicht:

In [ ]:
new_names_month_hour = {str(i): datetime(2023, i, 1).strftime('%B') for i in range(1,13)}
new_names_month_hour['hour'] = 'Stunde'
def formatter(v):
    if v < 1.0:
        return format(v*1000, '.2f') + " Wh"
    else:
        return format(v, '.2f') + " kWh"
    
df = db.execute('FROM v_average_production_per_month_and_hour') \
  .df().dropna(how='all', axis=1) \
  .rename(columns=new_names_month_hour) \
  .set_index(['Stunde'])
df.style \
  .background_gradient(cmap='YlOrRd') \
  .set_properties(**{'text-align': 'right'}) \
  .set_table_styles([dict(selector='th', props=[('text-align', 'right')])]) \
  .format(formatter) 

Die folgende Visualisierung zeigt ein Sonnenstandverlaufsdiagramm für den Ort unseres Hauses in Aachen. Zwei Dinge werden sehr schnell deutlich: Wir haben nur wenig Ertrag in den Wintermonaten, die Sonne wird zwischen 9 und 17 Uhr in einem für uns ungünstigen Winkel scheinen. Am kürzesten Tag des Jahres steht die Sonne am wahren Mittag genauso hoch wie sie es am längsten Tag des Jahres bereits um 8 Uhr morgens tut. Monate zwischen November und Februar werden sehr passend als "Tal der Tränen" für PV-Anlagen-Besitzer:innen beschrieben.

In der Darstellung befinden sich 4 Kurven: der Verlauf der Sonnen zur Winter- und Sommersonnenwende, der Verlauf am bisher produktivsten Tag im aktuellen Jahr sowie in Grau ein Tag zwischen den Sonnenwendtagen als Orientierung. Die hübschen, 8-förmigen Schleifen heissen [Analemma](https://de.wikipedia.org/wiki/Analemma). Diese Figur entsteht, wenn der Sonnenverlauf von einem fixen Punkt auf der Erde über ein Jahr täglich zur selben Zeit beobachten. Durch die Analemma können die Uhrzeiten auf den Verlaufskurven miteinander in Verbindung gebracht werden. 

In [ ]:
best_day = db.execute('FROM v_best_performing_days WHERE year = ?', [dt.year]).df()

times = pd.date_range(datetime(dt.year, 1, 1), datetime(dt.year+1, 1, 1), inclusive='left', freq='h', tz=tz)
solpos = pd.DataFrame(suncalc.get_position(times, long, lat))
solpos = pd.DataFrame(times).join(solpos, how='inner').loc[solpos['altitude'] > 0, :]
solpos = solpos.set_index(0)
solpos['altitude'] = np.vectorize(to_degree)(solpos['altitude'])
solpos['azimuth'] = np.vectorize(lambda v: math.degrees(v))(solpos['azimuth'])

fig, ax = plt.subplots(figsize=plt.figaspect(1/3))
points = ax.scatter(solpos['azimuth'], solpos['altitude'], s=2, c=solpos.index.dayofyear, label=None)
fig.colorbar(points)

for hour in np.unique(solpos.index.hour):
    subset = solpos.loc[solpos.index.hour == hour, :]
    height = subset['altitude']
    pos = solpos.loc[height.idxmax(), :]
    ax.text(pos['azimuth'], pos['altitude'], str(hour))
    
cmap = plt.colormaps.get_cmap('YlOrRd')
neutral_day = datetime(dt.year, 3, 21)
best_day_date = best_day['measured_on'][0].tz_localize(None).to_pydatetime()
dates_of_interest = [datetime(dt.year-1, 12, 21), best_day_date, neutral_day, datetime(dt.year, 6, 21)]
dates_of_interest.sort()

colors = prop_cycle.by_key()['color']
for index, date in enumerate(pd.to_datetime(dates_of_interest)):
    times = pd.date_range(date, date+pd.Timedelta('24h'), freq='1min', tz=tz)
    solpos = pd.DataFrame(suncalc.get_position(times, long, lat))
    solpos = solpos.loc[solpos['altitude'] > 0, :]
    solpos['altitude'] = np.vectorize(to_degree)(solpos['altitude'])
    solpos['azimuth'] = np.vectorize(lambda v: math.degrees(v))(solpos['azimuth'])
    label = date.strftime('%Y-%m-%d')
    color = 'grey' if date == neutral_day else colors[index]
    ax.plot(solpos['azimuth'], solpos['altitude'], label=label, color=color)
    
plt.legend(frameon=True, reverse=True)
ax.set_xlabel('Sonnenazimut in ° von Süden nach Westen')
ax.set_ylabel('Sonnenstand über dem Horizont in °')
plt.title(f"Sonnenverlauf in Aachen {dt.year}")

plt.show()

In [ ]:
Markdown(f"""
Der Sonnenverlauf sowie die Stärke der Sonneneinstrahlung in kWh/m² für den Tag mit der bisherigen höchsten Erzeugung in {dt.year} sehen so aus:
""")

In [ ]:
colors = prop_cycle.by_key()['color']

best_day['measured_on'] = best_day['measured_on'].apply(localize_tz)
best_day['altitude'] = pd.DataFrame(suncalc.get_position(best_day['measured_on'], long, lat)['altitude'].apply(to_degree))

_, ax = plt.subplots(figsize=plt.figaspect(1/3))

twin1 = ax.twinx()
twin1.spines.right.set_position(("axes", 1.05))

best_day.plot(x='measured_on', y='production', xlabel='Uhrzeit', ylabel='W', ax=ax, color=colors[0])
best_day.plot(x='measured_on', y='altitude', xlabel='Uhrzeit ', ylabel='Sonnenstand über dem Horizont in °', ax=ax, secondary_y=True, mark_right=False, color=colors[1])
best_day.plot(x='measured_on', y='shortwave_radiation', xlabel='Uhrzeit', ax=ax, color=colors[3])
best_day.plot(x='measured_on', y='cloud_coverage', xlabel='Uhrzeit', ylabel='Bedeckungsgrad in %', ax=twin1, mark_right=False, color=colors[2])

twin1.legend(handles=[], frameon=False)

with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    # hourAngle in suncalc.py will produce a NaN in acos, same as suncalc.js
    # the sun_times we want are correct and in line with suncalc.js, night and nightEnd will are NaT (not a time)
    sun_times = suncalc.get_times(best_day['measured_on'][0].floor('d') + pd.Timedelta(days=1), long, lat)
sun_times = {k: v.tz_localize(tz='UTC').tz_convert(tz='Europe/Berlin') for k, v in sun_times.items()}

ax.xaxis.set_major_formatter(lambda v, p: '')
ax.axvspan(sun_times['dawn'], sun_times['sunrise_end'], color='red', alpha=0.1)
ax.axvspan(sun_times['solar_noon'] - pd.Timedelta(minutes=5), sun_times['solar_noon'] + pd.Timedelta(minutes=5), color='yellow', alpha=0.4)
ax.axvspan(sun_times['sunset_start'], sun_times['dusk'], color='blue', alpha=0.1)

ax.legend(handles=[
    mlines.Line2D([], [], color=colors[0], label='Erzeugung'),
    mlines.Line2D([], [], color=colors[1], label='Sonnenstand'),
    mlines.Line2D([], [], color=colors[3], label='Sonneneinstrahlung je m²'),
    mlines.Line2D([], [], color=colors[2], label='Bedeckungsgrad'),
],loc="upper right")

plt.title(f"Erzeugung am {best_day['measured_on'][0].strftime('%x')}")
plt.legend([], frameon=False)
plt.show()

In [ ]:
Markdown(f"""
Die orangene Kurve gibt den Sonnenverlauf am {best_day['measured_on'][0].strftime('%x')} wieder, 
der rote Balken markiert die Zeit von Anfang der Morgendämmerung bis zum Ende des Sonnenaufgangs an diesem Tag,
der gelbe eine Phase von 10 Minuten um den wahren Mittag (dem Sonnenhöchststand) herum und
der blaue schlussendlich die Zeit vom Sonnenuntergang bis Ende der Abenddämmerung.
""")

In [ ]:
first_proper_readings = db.execute("SELECT value FROM domain_values WHERE name = 'FIRST_PROPER_READINGS_ON'").df()
Markdown(f"""
### Eigenverbrauchsanteil, Autarkiegrad und Amortisierung

*Achtung*: Aufgrund von Problemen mit dem Metering beginnen alle Statistiken die Verbrauch, Bezug und Einspeisung benutzen erst am **{datetime.strptime(first_proper_readings['value'][0],'%Y-%m-%d').strftime('%x')}**.
""")

#### Eigenverbrauchsanteil und Autarkiegrad

Klarsolar erklärt den Unterschied zwischen Eigenverbrauch und Autorkiegrad ganz schön [hier](https://klarsolar.de/unterschied-eigenverbrauch-autarkie/):
* Der Eigenverbrauchsanteil ist die Menge des selbst produzierten Solarstroms, die man direkt selbst verbraucht.
  <br>Er berechnet sich so: `(Erzeugung - Einspeisung) / Erzeugung * 100`
* Der Autarkiegrad beschreibt die Menge des Eigenverbrauchanteils im Verhältnis zum Gesamtstromverbrauch.
  <br>Er berechnet sich so: `(Erzeugung – Einspeisung) / Verbrauch * 100`


Unsere Quoten sehen aktuell—ohne große Optimierungen—so aus:

In [ ]:
yearly_internal_consumption_share

Unser Ziel ist mittelfristig die Erhöhung des Eigenverbrauchgrades, weniger die absolute Autarkie. Auch hier finde ich die Darstellung pro Stunde aufschlussreich, zumindest hinsichtlich wie wir unser eigenes Verhalten optimieren können:

In [ ]:
df = db.execute('SELECT hour, autarchy, internal_consumption FROM v_average_internal_consumption_share_per_hour').df()
df = df.rename(columns=new_names_quotes)
df.plot(kind='bar', x='hour', xlabel='Stunde', ylabel='%', figsize=plt.figaspect(1/3))

plt.title("Durchschnittlicher Autarkiegrad und Eigenverbrauchsanteil je Stunde")

plt.show()

Die folgende Grafik stellt das Verhältnis von Einspeisung und Eigenverbrauch der Gesamtproduktion zusammen mit dem Gesamtverbrauch pro Tag im laufenden Jahr da. Die Größe der roten Fläche, die dem Bezug entspricht, ist in dieser Ansicht umgekehrt proportional zur Grad der Autorkie.

In [ ]:
end = dt + pd.DateOffset(years=1) - pd.Timedelta(days=1)
df = db.execute('FROM v_energy_flow_per_day WHERE day BETWEEN ? and ?', [dt, end]).df()
df = df.set_index('day')

_, ax = plt.subplots(figsize=plt.figaspect(1/3))
c=cycler(color=['#8dd3c7', '#fb8072', '#ffffb3', '#bebada'])
ax.set_prop_cycle(c)

df['production'] = (df['production'] - df['consumption'] + df['import']).clip(lower=0)
df['consumption'] = df['internal_consumption'] + df['import']

df.loc[:,['internal_consumption','import', 'production']].plot(kind='area', stacked=True, ax=ax)
df.loc[:,['consumption']].plot(linewidth=2, ax=ax)

ax.xaxis.set_minor_formatter(lambda x,p: '')
ax.xaxis.set_major_formatter(mdates.DateFormatter('%b'))
h, l = ax.get_legend_handles_labels()
ax.legend(h, ["Eigenverbrauch", "Bezug", "Einspeisung", "Gesamtverbrauch"])
plt.xlabel("")
plt.ylabel("kWh")
plt.title(f"Verhältnis von Einspeisung und Eigenverbrauch in {dt.year}")

plt.show()

#### Amortisierung

Bei der Volleinspeisung wird die gesamte von einer Photovoltaikanlage erzeugte elektrische Energie ins öffentliche Stromnetz eingespeist, anstatt sie (teilweise) für den Eigenverbrauch zu nutzen. Bei der Teileinspeisung hingegen wird ein Teil der von einer Photovoltaikanlage erzeugten Energie ins öffentliche Stromnetz eingespeist. Der verbleibende Teil wird für den Eigenverbrauch im Haushalt oder Gebäude verwendet. Die Vergütung für die Volleinspeisung ist nur noch marginal höher als für Teileinspeisung und lohnt sich nicht mehr für Anlagen in der Größenordnung wie hier. Unsere Bezugskosten sind mindestens um den Faktor 4 höher als die Einspeisevergütung.

Die Vergütungen für eine Anlage unserer Größe in Teileinspeisung werden unten dargestellt:

In [ ]:
db.execute("SELECT valid_from as 'Gültig von', valid_until as 'Gültig bis', value as 'ct/kWh' FROM v__selling_prices WHERE type = 'partial_sell'").df().fillna('-')

Das folgende Diagram präsentiert die laufende Summe unser initialen Investitionskosten plus Vergütung und Ersparnissen.

Die Amortisierung bei Volleinspeisung ist hypothetisch und basiert auf der reinen Vergütung.
Die Amortisierung bei Teileinspeisung basiert auf der Vergütung der eingespeisten Energie plus der Ersparniss durch den Eigenverbrauch (aka dem nicht Kaufen von Energie).

Alle Werte in der mit hellem Lila hinterlegte Fläche sind interpolierte Werte. Allerdings ist doch recht offensichtlich, dass sich eine in 2023 angeschaffte Anlege nicht mehr in derselben Dekade amortisieren wird.

In [ ]:
# Idea of the linear extrapolation is mostly from this great answer by tmthydvnprt
# https://stackoverflow.com/a/35959909/1547989

df = db.execute('SELECT month, full_export, partial_export FROM v_amortization').df()
df.set_index(['month'], inplace=True)

_, ax = plt.subplots(figsize=plt.figaspect(1/3))

legend = ['Hypothetische Volleinspeisung', 'Teileinspeisung', 'Break even']
# Memorize last entry
last_month = df.index[len(df)-1]
first_extrapolated_month = None

if len(df) < 120:    
    num_months_extrapolation = 120 - len(df)
    
    # Extend the index
    df = pd.DataFrame(data=df, index=pd.date_range(
            start=df.index[0],
            periods=len(df.index) + num_months_extrapolation,
            freq='MS'
        )
    )
    
    first_extrapolated_month = last_month + pd.DateOffset(months=1);
    
    di = df.index
    df = df.reset_index(drop=True)
    
    # Function to curve fit to the data
    def func(x, a, b):
        return a * x + b
    
    # Initial parameter guess, just to kick off the optimization
    guess = (1, 1)
    
    # Create copy of data to remove NaNs for curve fitting
    fit_df = df.dropna()
    
    # Place to store function parameters for each column
    col_params = {}
    
    # Curve fit each column
    for col in fit_df.columns:
        # Get x & y
        x = fit_df.index.astype(float).values
        y = fit_df[col].values
        with warnings.catch_warnings():
            warnings.simplefilter('ignore')
            # Curve fit column and get curve parameters
            params = curve_fit(func, x, y, guess)
            # Store optimized parameters
            col_params[col] = params[0]
    
    # Extrapolate each column
    for col in df.columns:
        # Get the index values for NaNs in the column
        x = df[pd.isnull(df[col])].index.astype(float).values
        # Extrapolate those points with the fitted function
        df.loc[x, col] = func(x, *col_params[col])
    
    df.index = di
    legend = legend + ['Tatsächliche Daten', 'Extrapolierte Werte']


df.plot(ax=ax)
ax.axhline(color='green')
if first_extrapolated_month != None:
    ax.axvspan(df.index[0], first_extrapolated_month, color='green', alpha=0.1)
    ax.axvspan(first_extrapolated_month, df.index[-1], color='purple', alpha=0.05)
ax.legend(legend , loc='lower right')

plt.xlabel('Zeit in Betrieb')
plt.ylabel('EUR')
plt.ylim(top=5000)
plt.title("Amortisierungsverlauf")

plt.show()

Photovoltaik in Deutschland mit einer Anlage dieser Größe ist kein Mittel um schnell reich zu werden. Eher, um langsamer arm zu werden wie mein Freund Oli es ausdrückte. Daher zum Abschluss wieder ein ähnliches Diagram wie in seinem Blog. Das Diagram zeigt die jährlich akkumulierten Energiekosten in unserem Haus. Die erste Kurve ohne den Nutzen und die Ersparnisse der Photovoltaikanlage zu betrachten, die zweite Kurve bilanziert die tatsächlich angefallen Verbrauchskosten mit den Ersparnissen und Vergütungen. Falls wir am Ende eines vollen Jahres die null mit dieser Kurve erreichen, bin ich zufrieden. Negative Werte bedeuten entsprechen einer realen Vergütung.

In [ ]:
df = db.execute('FROM v_accumulated_yearly_energy_costs WHERE year(month) = ?', [dt.year]).df()

_, ax = plt.subplots(figsize=plt.figaspect(1/3))

colors = {1: 'tab:red', 0: 'black', -1: 'tab:green'}

def color(val):
    return (colors.get(int(np.sign(val))))

def labels(row):
    if row['cost_without_pv'] > 0: ax.text(x=row['month'], y=row['cost_without_pv'], s=row['cost_without_pv'], color=color(row['cost_without_pv']))
    ax.text(x=row['month'], y=row['cost_with_pv'], s=row['cost_with_pv'], color=color(row['cost_with_pv']))

df.plot(x='month', ax = ax, xlabel='Monat', ylabel='EUR')
df.apply(axis=1, func=labels);
ax.axhline(color='green', linestyle="--")
ax.legend(['Hypothetische Energiekosten ohne PV', 'Tatsächliche Stromkosten', "Hehres Ziel"])

l, r = plt.xlim()
plt.xlim(l-.5, r+.5)
plt.title(f"Akkumulierte Energiekosten in {dt.year}")

plt.show()

#### Verbrauch

Mein Leben lang habe ich ohne einen niedrigschwellig ablesbaren Stromzähler gelebt.
Der Mensch läuft ja nicht alle paar Minuten in den Keller, um den ["Ferraris-Zähler"](https://de.wikipedia.org/wiki/Ferraris-Zähler) abzulesen. Da wir aber tatsächlich für einen IT-Haushalt außergewöhnlich wenige dauerhaft laufende Dinge hier rumstehen haben, war der Verbrauch für eine vierköpfige Familie eigentlich immer im Rahmen.

Nichts desto trotz ist es spannend, die eigenen Muster zu sehen und vielleicht doch an der einen oder anderen Stelle zu hinterfragen. Wir haben kurz überlegt, noch einige Smart-Steckdosen anzuschaffen und über den Energymanager zu steuern und einige Verbräuche zu optimieren, aber das widerspräche zum einen dem Wunsch eben möglichst wenige, crappy "Smart-Devices" hier zu haben und zum anderen: Wieviel Strom muss ich mit einer Wifi-Steckdose sparen, bis sich 40€ oder mehr plus der Stromverbrauch von dem Dingen rechtfertigen?
Moderne Kühl- und Gefrierschränke nachts abzuschalten lohnt ebenfalls kaum, die Motoren laufen wenig… Das ist schön in den aufgezeichneten Verbrauchskurven sichtbar.

Am Ende haben wir uns dafür entschieden, einige Verbraucher von denen wir sicher wissen, dass sie nachts nicht im Standby rumidlen müssen, mit einer [ernsthaft smarten Steckdose](https://de.wikipedia.org/wiki/Zeitschaltuhr#Mechanische_Zeitschaltuhren) auszustatten und gut ist… Mit einer Grundlast von 100 bis 200 Watt kann ich ohne Bauchschmerzen leben.

In [ ]:
df = db.execute('FROM v_average_consumption_per_month_and_hour') \
  .df().dropna(how='all', axis=1) \
  .rename(columns=new_names_month_hour) \
  .set_index(['Stunde'])
df.style \
  .background_gradient(cmap='plasma') \
  .set_properties(**{'text-align': 'right'}) \
  .set_table_styles([dict(selector='th', props=[('text-align', 'right')])]) \
  .format(formatter) 

## Raincity Aachen?

In [ ]:
min_max_w = db.execute('SELECT min(ref_date) AS min, max(ref_date) FROM daily_weather_data').fetchone()

Markdown(f"""
Dank [Open Meteo](https://open-meteo.com) ist es tatsächlich kein Problem, Standort genaue, historische Wetterdaten zu bekommen und die Frage zu beantworten, ob Aachen wirklich dem Ruf "Raincity Aachen ☔️" gerechet wird.
Die folgende Grafik basiert auf täglichen Wetterdaten vom {min_max_w[0].strftime('%x')} bis {min_max_w[1].strftime('%x')}:
""")

In [ ]:
colors = prop_cycle.by_key()['color']
monthly_weather = db.execute('FROM v_monthly_weather').df()
monthly_weather['month'] = monthly_weather['month'].apply(lambda i: datetime(2024, i, 1).strftime('%B'))

fig, ax = plt.subplots(figsize=plt.figaspect(1/3))
twin1 = ax.twinx()
twin1.spines.right.set_position(("axes", 1.05))

monthly_weather.plot(xlabel='Monat', x='month', y='average_sunshine_duration_h', ylabel='h', kind='bar', ax=ax, color=colors[0])
monthly_weather.plot(xlabel='Monat', x='month', y='average_precipitation_mm', ylabel='mm', ax=ax, secondary_y=True, mark_right=False, color=colors[1])
monthly_weather.plot(xlabel='Monat', x='month', y='average_temperature', ax=twin1, mark_right=False, color=colors[2])

twin1.set_ylabel("°C")
twin1.legend(handles=[], frameon=False)

blue_line = mlines.Line2D([], [], color=colors[1], label='Niederschlag')
ax.legend(handles=[
    mpatches.Patch(color=colors[0], label='Sonnenstunden'), 
    mlines.Line2D([], [], color=colors[1], label='Niederschlag'),
    mlines.Line2D([], [], color=colors[2], label='Temperatur'),
])

plt.legend([], frameon=False)
plt.title("Historische Wetterdaten am Standort")
plt.show()

In [ ]:
def is_daytime(day, long, lat):
    day = day.to_pandas().rename('day')
    with warnings.catch_warnings():
        warnings.simplefilter('ignore')
        sun_times = pd.DataFrame(suncalc.get_times(day, long, lat))
        d = pd.concat([day, sun_times], axis=1)
        return d.apply(lambda row: row['day'] >= row['sunrise'] and row['day'] <= row['sunset'], axis=1)

with warnings.catch_warnings():
    warnings.simplefilter('ignore', category=DeprecationWarning) # Necessary due to DuckDBs interaction with numpy.
    db.create_function("is_daytime", is_daytime, [TIMESTAMP_TZ, DOUBLE, DOUBLE], BOOLEAN, type="arrow")

t = db.execute("""
  WITH averages AS (  
    SELECT datetrunc('day', measured_on) AS day,     
           avg(temperature_2m) FILTER (is_daytime(measured_on AT TIME ZONE 'Europe/Berlin', loc.long, loc.lat))     AS avg_day,
           avg(temperature_2m) FILTER (NOT is_daytime(measured_on AT TIME ZONE 'Europe/Berlin', loc.long, loc.lat)) AS avg_night,
    FROM v_place_of_installation loc, weather_data
    WHERE year(measured_on) = ? 
    GROUP BY ALL
  )
  SELECT round(min(avg_night), 2) AS min_avg_night, arg_min(day, avg_night) AS min_avg_night_at, 
         round(max(avg_night), 2) AS max_avg_night, arg_max(day, avg_night) AS max_avg_night_at,
         round(min(avg_day), 2) AS min_avg_day, arg_min(day, avg_day) AS min_avg_day_at, 
         round(max(avg_day), 2) AS max_avg_day, arg_max(day, avg_day) AS max_avg_day_at         
  FROM averages;
""", [dt.year]).df().to_dict(orient='index')[0]

Markdown(f"""
**{dt.year}** wurde die niedrigste, durchschnittliche Temperatur in der Nacht mit **{t['min_avg_night']}°C** am **{t['min_avg_night_at'].strftime('%d. %B')}** gemessen und die höchste mit  **{t['max_avg_night']}°C** am **{t['max_avg_night_at'].strftime('%d. %B')}**.
Die niedrigste, durchschnittliche Tagestemperatur betrug am **{t['min_avg_day_at'].strftime('%d. %B')}** **{t['min_avg_day']}°C** und die höchste **{t['max_avg_day']}°C** am **{t['max_avg_day_at'].strftime('%d. %B')}**
""")